In [1]:
import sys
import os
import numpy as np
import pandas as pd

import h5py

import gc


In [2]:
#Load scores and auxiliary data, compute mean over folds, and save new scores

fold_index = [0, 1, 2, 3]

#Initialize HDF5
scores_h5 = h5py.File('scores_mean.h5', 'w')

seqs = None
grads = None
genes = None
chrs = None
starts = None
ends = None
strands = None

#Loop over folds
for fold_i, fold_ix in enumerate(fold_index) :
    
    #Load score file
    score_file = h5py.File('scores_f' + str(fold_ix) + 'c0.h5', 'r')

    if fold_i == 0 :
        seqs = score_file['seqs'][()]
        grads = score_file['grads'][()]
        genes = score_file['gene'][()]
        chrs = score_file['chr'][()]
        starts = score_file['start'][()]
        ends = score_file['end'][()]
        strands = score_file['strand'][()]
    else :
        grads += score_file['grads'][()]
    
    #Collect garbage
    gc.collect()

grads /= float(len(fold_index))

#Re-save datasets in h5
scores_h5.create_dataset('seqs', data=np.array(seqs, dtype='bool'))
scores_h5.create_dataset('grads', data=np.array(grads, dtype='float16'))

scores_h5.create_dataset('gene', data=np.array(genes, dtype='S'))
scores_h5.create_dataset('chr', data=np.array(chrs, dtype='S'))
scores_h5.create_dataset('start', data=np.array(starts))
scores_h5.create_dataset('end', data=np.array(ends))
scores_h5.create_dataset('strand', data=np.array(strands, dtype='S'))

#Close h5
scores_h5.close()
